# Derivative-based methods

## Thanks and Credits
The core exercises are taken directly from [Daniel Newman's Github repository](https://github.com/dtnewman/stochastic_gradient_descent), which is distributed freely.

In [ ]:
%matplotlib inline
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
from scipy.optimize import fmin
plt.style.use('seaborn-white')
plt.rcParams.update({'font.size': 18})

### Gradient Descent

<b>Gradient descent</b>, also known as <b>steepest descent</b>, is an optimization algorithm for finding the local minimum of a function. To find a local minimum, the function "steps" in the  direction of the negative of the gradient. <b>Gradient ascent</b> is the same as gradient descent, except that it steps in the direction of the positive of the gradient and therefore finds local maximums instead of minimums. The algorithm of gradient descent can be outlined as follows:

&nbsp;&nbsp;&nbsp; 1: &nbsp; Choose initial guess $x_0$ <br>
&nbsp;&nbsp;&nbsp;    2: &nbsp; <b>for</b> k = 0, 1, 2, ... <b>do</b> <br>
&nbsp;&nbsp;&nbsp;    3:   &nbsp;&nbsp;&nbsp;&nbsp;&nbsp; $s_k$ = -$\nabla f(x_k)$ <br>
&nbsp;&nbsp;&nbsp;    4:   &nbsp;&nbsp;&nbsp;&nbsp;&nbsp; choose $\alpha_k$ to minimize $f(x_k+\alpha_k s_k)$ <br>
&nbsp;&nbsp;&nbsp;    5:   &nbsp;&nbsp;&nbsp;&nbsp;&nbsp; $x_{k+1} = x_k + \alpha_k s_k$ <br>
&nbsp;&nbsp;&nbsp;    6: &nbsp;  <b>end for</b>

As a simple example, let's find a local minimum for the function $f(x) = x^3-2x^2+2$

In [ ]:
def f(x) : return x**3 - 2.0*x**2 + 2.0
# An alternate way of doing the same thing:
# f = lambda x: x**3-2*x**2+2

In [ ]:
x = np.linspace(-1,2.5,1000)
plt.plot(x, f(x))
plt.xlabel('x')
plt.ylabel('f(x)')
plt.xlim([-1,2.5])
plt.ylim([0,3])
plt.show()

We can see from plot above that our local minimum is gonna be near around 1.4 or 1.5 (on the x-axis), but let's pretend that we don't know that, so we set our starting point (arbitrarily, in this case) at $x_0 = 2$

In [ ]:
x_old = 0
x_new = 2 # The algorithm starts at x=2
n_k = 0.1 # step size
precision = 0.0001

x_list, y_list = [x_new], [f(x_new)]

# returns the value of the derivative of our function
def f_prime(x):
    return 3*x**2-4*x
 
while abs(x_new - x_old) > precision:
    x_old = x_new
    s_k = -f_prime(x_old)
    x_new = x_old + n_k * s_k
    x_list.append(x_new)
    y_list.append(f(x_new))
print("Local minimum occurs at:", x_new)
print("Number of steps:", len(x_list))

The figures below show the route that was taken to find the local minimum.

In [ ]:
plt.figure(figsize=[10,3])
plt.subplot(1,2,1)
plt.plot(x,f(x))
plt.plot(x_list,y_list,"ro-", ms=12)
plt.xlim([-1,2.5])
plt.ylim([0,3])
plt.xlabel('x')
plt.ylabel('f(x)')
plt.title("Gradient descent")
plt.subplot(1,2,2)
plt.plot(x,f(x))
plt.plot(x_list,y_list,"ro-", ms=12)
plt.xlim([1.2,2.1])
plt.ylim([0,3])
plt.xlabel('x')
plt.ylabel('f(x)')
plt.title("Gradient descent (zoomed in)")
plt.show()

You'll notice that the step size (also called learning rate) in the implementation above is constant, unlike the algorithm in the pseudocode. Doing this makes it easier to implement the algorithm. However, it also presents some issues: If the step size is too small, then convergence will be very slow, but if we make it too large, then the method may fail to converge at all. 

A solution to this is to use adaptive step sizes as the algorithm below does (using `scipy`'s `fmin` function to find optimal step sizes):

In [ ]:
# we setup this function to pass into the fmin algorithm
def f2(n,x,s):
    x = x + n*s
    return f(x)

x_old = 0
x_new = 2 # The algorithm starts at x=2
precision = 0.0001

x_list, y_list = [x_new], [f(x_new)]

# returns the value of the derivative of our function
def f_prime(x):
    return 3*x**2-4*x

while abs(x_new - x_old) > precision:
    x_old = x_new
    s_k = -f_prime(x_old)
    
    # use scipy fmin function to find ideal step size.
    n_k = fmin(f2,0.1,(x_old,s_k), full_output = False, disp = False)

    x_new = x_old + n_k * s_k
    x_list.append(x_new)
    y_list.append(f(x_new))
    
print("Local minimum occurs at ", float(x_new))
print("Number of steps:", len(x_list))

With adaptive step sizes, the algorithm converges in just 4 iterations rather than 17. Of course, it takes time to compute the appropriate step size at each iteration. Here are some plots of the path taken below. You can see that it converges very quickly to a point near the local minimum, so it's hard to even discern the dots after the first two steps until we zoom in very close in the third frame below:

In [ ]:
plt.figure(figsize=[15,3])
plt.subplot(1,3,1)
plt.plot(x,f(x))
plt.plot(x_list,y_list,"ro-", ms=12)
plt.xlim([-1,2.5])
plt.xlabel('x')
plt.ylabel('f(x)')
plt.title("Gradient descent")
plt.subplot(1,3,2)
plt.plot(x,f(x))
plt.plot(x_list,y_list,"ro-", ms=12)
plt.xlim([1.2,2.1])
plt.ylim([0,3])
plt.xlabel('x')
plt.ylabel('f(x)')
plt.title("zoomed in")
plt.subplot(1,3,3)
plt.plot(x,f(x))
plt.plot(x_list,y_list,"ro-", ms=12)
plt.xlim([1.333,1.334])
plt.ylim([0.814,0.816])
plt.xlabel('x')
#plt.ylabel('f(x)')
plt.title("zoomed in more")
plt.show()

Another approach to update the step size is choosing a decrease constant $d$ that shrinks the step size over time:
$\eta(t+1) = \eta(t) / (1+t \times d)$. This is commonly done in supervised machine-learning methods (where a variation of steepest descent called the Stochastic Gradient Descent (SGD) is used).

In [ ]:
x_old = 0
x_new = 2 # The algorithm starts at x=2
n_k = 0.17 # step size
precision = 0.0001
t, d = 0, 1

x_list, y_list = [x_new], [f(x_new)]

# returns the value of the derivative of our function
def f_prime(x):
    return 3*x**2-4*x
 
while abs(x_new - x_old) > precision:
    x_old = x_new
    s_k = -f_prime(x_old)
    x_new = x_old + n_k * s_k
    x_list.append(x_new)
    y_list.append(f(x_new))
    n_k = n_k / (1 + t * d)
    t += 1

print("Local minimum occurs at:", x_new)
print("Number of steps:", len(x_list))

### Gradient Descent in two-dimensions

The same algorithm works independent of the dimensions! The derivatives are now gradients and hence vectors...

In [ ]:
x_old = np.array([0.0, 0.0])
x_new = np.array([6.0, 6.0]) # The algorithm starts at x=2
n_k = 0.1 # step size
precision = 0.0001
t, d = 0, 1

stretch_factor = 10
def f(x):
    return x[0]**2 + stretch_factor * x[1]**2

# returns the value of the derivative of our function
def f_prime(x):
    return np.array([2.0*x[0], 2.0*stretch_factor*x[1]])

def f2(n,x,s):
    x = x + n*s
    return f(x)

x_list, y_list = [x_new], [f(x_new)]

while np.linalg.norm(x_new - x_old) > precision:
    x_old = x_new
    s_k = -f_prime(x_old)
    # use scipy fmin function to find ideal step size.
    # n_k = fmin(f2,0.1,(x_old,s_k), full_output = False, disp = False)
    x_new = x_old + n_k * s_k
    x_list.append(x_new)
    y_list.append(f(x_new))
    #n_k = n_k / (1 + t * d)
    #t += 1

print("Local minimum occurs at:", x_new)
print("Number of steps:", len(x_list))

In [ ]:
fig = plt.figure(figsize=(8,8))
ax = fig.add_subplot(111)
x_collection = np.array(x_list)
x_collection = x_collection if x_collection.shape[1] == 2 else x_collection.T
ax.plot(x_collection[:, 0], x_collection[:, 1], 'ro-', ms=14)
grid_x = np.linspace(-6.0, 6.0, 100)
grid_y = np.linspace(-6.0, 6.0, 100)
X,Y = np.meshgrid(grid_x, grid_y)
Z = f([X, Y])
ax.contourf(X, Y ,Z, cmap=plt.cm.viridis)
ax.set_xlabel('x')
ax.set_ylabel('y')
ax.set_title('f(x,y)')
ax.set_aspect('equal')

### Brittle
But it's very easy to break. Try changing the `stretch_factor` in the example above. The conjugate gradient method overcomes this difficulty with `stretch_factor`.

## Method of Conjugate Gradients

If we need to minimize a function of the form

$$ \mathbf{x}^* = \textrm{argmin} \left( {\tfrac {1}{2}} \mathbf{x}^{\mathsf {T}} \mathbf{A} \mathbf{x} - \mathbf{x}^{\mathsf {T}}\mathbf{b} \right) $$

which reduces to solving $ \mathbf{A} \mathbf{x} - \mathbf{b} = 0$, we can use the following algorithm (found [here](https://en.wikipedia.org/wiki/Conjugate_gradient_method#The_resulting_algorithm)). An approachable introduction to understand CG can be found in this [link](http://www.cs.cmu.edu/~quake-papers/painless-conjugate-gradient.pdf).

\begin{aligned}&\mathbf {r} _{0}:=\mathbf {b} -\mathbf {Ax} _{0}\\&{\hbox{if }}\mathbf {r} _{0}{\text{ is sufficiently small, then return }}\mathbf {x} _{0}{\text{ as the result}}\\&\mathbf {p} _{0}:=\mathbf {r} _{0}\\&k:=0\\&{\text{repeat}}\\&\qquad \alpha _{k}:={\frac {\mathbf {r} _{k}^{\mathsf {T}}\mathbf {r} _{k}}{\mathbf {p} _{k}^{\mathsf {T}}\mathbf {Ap} _{k}}}\\&\qquad \mathbf {x} _{k+1}:=\mathbf {x} _{k}+\alpha _{k}\mathbf {p} _{k}\\&\qquad \mathbf {r} _{k+1}:=\mathbf {r} _{k}-\alpha _{k}\mathbf {Ap} _{k}\\&\qquad {\hbox{if }}\mathbf {r} _{k+1}{\text{ is sufficiently small, then exit loop}}\\&\qquad \beta _{k}:={\frac {\mathbf {r} _{k+1}^{\mathsf {T}}\mathbf {r} _{k+1}}{\mathbf {r} _{k}^{\mathsf {T}}\mathbf {r} _{k}}}\\&\qquad \mathbf {p} _{k+1}:=\mathbf {r} _{k+1}+\beta _{k}\mathbf {p} _{k}\\&\qquad k:=k+1\\&{\text{end repeat}}\\&{\text{return }}\mathbf {x} _{k+1}{\text{ as the result}}\end{aligned}

We can couch the problems seen above, of minimizing $x^2 + \texttt{stretch_factor} * y^2$ into the following form:

\begin{equation*}
\mathbf{x}^* = \textrm{argmin} \left( {\tfrac {1}{2}} \mathbf{x}^{\mathsf {T}} \cdot \begin{bmatrix}
1 & 0\\
0 & \texttt{stretch_factor}
\end{bmatrix}
\cdot \mathbf{x} - \mathbf{x}^{\mathsf {T}}
\begin{bmatrix}
0 \\
0
\end{bmatrix}\right) \\
\end{equation*}


In [ ]:
stretch_factor = 100.0
A = np.array([[1.0, 0.0], [0.0, stretch_factor]])
b = np.zeros((2,))

In [ ]:
x = np.array([6.0, 6.0])
x_list = [x]
i = 0
imax = 10 # max number of iterations
eps = 0.0001
r = b - A@x
d = r
deltanew = np.inner(r, r)
delta0 = deltanew
while i < imax and deltanew > eps**2 * delta0:
    alpha = float(deltanew / np.inner(d , (A @ d)))
    x = x + alpha * d
    x_list.append(x)
    r = b - A @ x
    deltaold = deltanew
    deltanew = np.inner(r, r)
    #beta = -float((r.T * A * d) / float(d.T * A * d))
    beta = float(deltanew / float(deltaold))
    d = r + beta * d
    i += 1

In [ ]:
fig = plt.figure(figsize=(8,8))
ax = fig.add_subplot(111)
x_collection = np.array(x_list)
x_collection = x_collection if x_collection.shape[1] == 2 else x_collection.T
ax.plot(x_collection[:, 0], x_collection[:, 1], 'ro-', ms=14)
grid_x = np.linspace(-6.0, 6.0, 100)
grid_y = np.linspace(-6.0, 6.0, 100)
X,Y = np.meshgrid(grid_x, grid_y)
Z = f([X, Y])
ax.contourf(X, Y ,Z, cmap=plt.cm.viridis)
ax.set_aspect('equal')

## Is this realistic?

That's great, but how useful is it in real-life functions that are
- Multi-modal (the above was a unimodal function, with one global minima)
- Non-convex (the above was a convex function)
- Non-separable (in the above example x and y are equivalent but separate)
- Non-linear (the above problem is essentially linear)

? 

To test that, let's take the Rastrigin function that was discussed a couple of lectures ago and apply steepest descent and CG to minimize it. We need to locally linearize the problem at every step, which involves finding gradients (first-derivatives : a vector) and Hessians (second-derivatives : a matrix) of the function! The rastrigin function in two dimensions is :
$$f(\mathbf{x}) = 20 + \left[ x^2 - 10 \cos\left(2 \pi x \right) \right] + \left[ y^2 - 10 \cos\left(2 \pi y \right) \right]$$

The gradient is :
$$ \nabla f(\mathbf{x}) = \begin{bmatrix}
2x + 20 \pi \sin\left(2 \pi x \right) \\
2y + 20 \pi \sin\left(2 \pi y \right) 
\end{bmatrix}
$$

and finally the Hessian 
$$ \nabla^2 f(\mathbf{x}) = \begin{bmatrix}
2 + 40 \pi^2 \cos\left(2 \pi x \right) &  0\\
0 & 2 + 40 \pi^2 \cos\left(2 \pi x \right)
\end{bmatrix}
$$


In [ ]:
x = np.array([4.0, 3.0])
x_list = [x]

i = 0
imax = 10 # max number of iterations
eps = 0.0001

def f(x):
    return 20.0 + ((x[0]-2.0)**2 - 10.0 * np.cos(2.0 * np.pi * (x[0]-2.0))) + ((x[1]-2.0)**2 - 10.0 * np.cos(2.0 * np.pi * (x[1]-2.0)))

def grad_f(x):
    return np.array([2.0 * (x[0]-2.0) + 20.0 * np.pi * np.sin(2.0 * np.pi * (x[0]-2.0)),
                     2.0 * (x[1]-2.0) + 20.0 * np.pi * np.sin(2.0 * np.pi * (x[1]-2.0))])

def hessian(x):
    return np.array([[2.0 + 40.0 * np.pi**2 * np.cos(2.0 * np.pi * (x[0]-2.0)), 0.0],
                     [0.0, 2.0 + 40.0 * np.pi**2 * np.cos(2.0 * np.pi * (x[1]-2.0))]])


r = grad_f(x) - hessian(x)@x
d = r
deltanew = np.inner(r, r)
delta0 = deltanew

while i < imax and deltanew > eps**2 * delta0:
    alpha = float(deltanew / np.inner(d , (A @ d)))
    x = x + alpha * d
    A = hessian(x)
    b = grad_f(x)
    x_list.append(x)
    r = b - A @ x
    deltaold = deltanew
    deltanew = np.inner(r, r)
    beta = float(deltanew / float(deltaold))
    d = r + beta * d
    i += 1

In [ ]:
fig = plt.figure(figsize=(8,8))
ax = fig.add_subplot(111)
x_collection = np.array(x_list)
ax.plot(x_collection[:, 0], x_collection[:, 1], 'ro-', ms=14)
grid_x = np.linspace(-5.0, 5.0, 100)
grid_y = np.linspace(-5.0, 5.0, 100)
X,Y = np.meshgrid(grid_x, grid_y)
Z = f([X, Y])
ax.contourf(X, Y ,Z, cmap=plt.cm.viridis)
ax.set_aspect('equal')